In [1]:

import requests
import pandas as pd

url = "https://www.nike.com/es/w?q=metcon&vst=metcon"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.google.com'
}
response = requests.get(url,headers=headers)
from bs4 import BeautifulSoup
soup=BeautifulSoup(response.content, "html.parser")

In [2]:
"""import pandas as pd
from bs4 import BeautifulSoup

# Crear DataFrame vacío con las columnas deseadas
df = pd.DataFrame(columns=['title', 'price'])

# Extraer todas las tarjetas de producto
product_titles = soup.find_all('div', class_='product-card__title')
product_prices = soup.find_all('div', class_='product-card__price')

# Asegurarse de que la longitud de las listas coincida
num_items = min(len(product_titles), len(product_prices))

# Iterar sobre las tarjetas y extraer información
for i in range(num_items):
    # Obtener el título del producto
    title = product_titles[i].text.strip() if product_titles[i] else "N/A"
    # Obtener el precio del producto
    price = product_prices[i].text.strip() if product_prices[i] else "N/A"
    # Agregar al DataFrame
    df.loc[i] = [title, price]

# Mostrar el DataFrame resultante
print(df)"""

'import pandas as pd\nfrom bs4 import BeautifulSoup\n\n# Crear DataFrame vacío con las columnas deseadas\ndf = pd.DataFrame(columns=[\'title\', \'price\'])\n\n# Extraer todas las tarjetas de producto\nproduct_titles = soup.find_all(\'div\', class_=\'product-card__title\')\nproduct_prices = soup.find_all(\'div\', class_=\'product-card__price\')\n\n# Asegurarse de que la longitud de las listas coincida\nnum_items = min(len(product_titles), len(product_prices))\n\n# Iterar sobre las tarjetas y extraer información\nfor i in range(num_items):\n    # Obtener el título del producto\n    title = product_titles[i].text.strip() if product_titles[i] else "N/A"\n    # Obtener el precio del producto\n    price = product_prices[i].text.strip() if product_prices[i] else "N/A"\n    # Agregar al DataFrame\n    df.loc[i] = [title, price]\n\n# Mostrar el DataFrame resultante\nprint(df)'

In [49]:
import pandas as pd
from bs4 import BeautifulSoup
import time
import requests
import re

def clean_price(price_str):
    """
    Limpia la cadena de precio, eliminando símbolos no numéricos.
    
    Args:
        price_str (str): Cadena con el precio a limpiar.
    
    Returns:
        float: Precio convertido a número.
    """
    # Eliminar símbolos de moneda, espacios especiales y reemplazar coma por punto
    clean_str = re.sub(r'[^\d,.]', '', price_str)
    
    # Reemplazar coma por punto si es un separador decimal
    clean_str = clean_str.replace(',', '.')
    
    try:
        return float(clean_str)
    except ValueError:
        return float('inf')  # Valor muy alto para comparaciones

def extract_product_data(soup):
    """
    Extrae información de los productos, incluyendo títulos, precios, links, descripción y materiales.

    Args:
        soup (BeautifulSoup): Objeto BeautifulSoup con el HTML parseado.

    Returns:
        pd.DataFrame: DataFrame con las columnas 'title', 'current_price', 'discount_price',
                      'description', 'materials', 'link'.
    """
    # Crear DataFrame vacío con las columnas deseadas
    df = pd.DataFrame(columns=['title', 'current_price', 'discount_price', 'description', 'materials', 'link'])

    # Extraer todas las tarjetas de producto con los nuevos selectores
    product_titles = soup.find_all('div', class_='product-card__title')
    product_price_containers = soup.find_all('div', class_='product-card__price')
    product_descriptions = soup.find_all('div', class_='product-card__subtitle')
    product_materials = soup.find_all('div', class_='product-card__messaging')
    product_links = soup.find_all('a', class_='product-card__link-overlay')

    # Determinar el número de productos (basado en títulos)
    num_items = len(product_titles)

    # Iterar sobre las tarjetas de productos y extraer información
    for i in range(num_items):
        # Pausa para evitar ser bloqueado por el servidor
        time.sleep(1)

        # Obtener el título del producto
        title = product_titles[i].text.strip() if i < len(product_titles) else "N/A"

        # Obtener los precios
        current_price = "N/A"
        discount_price = "N/A"
        
        if i < len(product_price_containers):
            price_wrapper = product_price_containers[i].find('div', class_='product-price__wrapper')
            
            if price_wrapper:
                # Buscar elementos de precio de diferentes clases
                price_elems = price_wrapper.find_all('div', class_=['es__styling', 'is--current-price', 'is--original-price'])
                
                # Filtrar y limpiar precios
                prices = [elem.text.strip() for elem in price_elems if elem and elem.text.strip()]
                
                # Lógica de asignación de precios
                if len(prices) >= 2:
                    # Ordenar precios de menor a mayor
                    sorted_prices = sorted(prices, key=clean_price)
                    current_price = sorted_prices[0]
                    discount_price = sorted_prices[-1]
                elif len(prices) == 1:
                    current_price = prices[0]
                    discount_price = current_price

        # Obtener la descripción del producto
        description = product_descriptions[i].text.strip() if i < len(product_descriptions) else "N/A"

        # Obtener los materiales
        materials = product_materials[i].text.strip() if i < len(product_materials) else "N/A"

        # Obtener el link del producto
        link = product_links[i]['href'] if i < len(product_links) else "N/A"

        # Agregar los datos al DataFrame
        df.loc[i] = [title, current_price, discount_price, description, materials, link]

    return df

In [50]:
extract_product_data(soup)

,title,current_price,discount_price,description,materials,link
0,Nike Free Metcon 6,"103,99 €","129,99 €",Zapatillas de training - Mujer,Materiales sostenibles,https://www.nike.com/es/t/free-metcon-6-zapati...
1,Nike Free Metcon 6,"129,99 €","129,99 €",Zapatillas de training - Hombre,Materiales sostenibles,https://www.nike.com/es/t/free-metcon-6-zapati...
2,Nike Metcon 9,"97,99 €","139,99 €",Zapatillas de training - Mujer,Materiales sostenibles,https://www.nike.com/es/t/metcon-9-zapatillas-...
3,Nike Metcon 9,"97,99 €","139,99 €",Zapatillas de training - Hombre,Materiales sostenibles,https://www.nike.com/es/t/metcon-9-zapatillas-...
4,Nike Metcon 9,"139,99 €","139,99 €",Zapatillas de training - Hombre,Materiales sostenibles,https://www.nike.com/es/t/metcon-9-zapatillas-...
5,Nike Metcon 9 AMP,"149,99 €","149,99 €",Zapatillas de training - Mujer,Materiales sostenibles,https://www.nike.com/es/t/metcon-9-amp-zapatil...
6,Nike Metcon 1 OG,"104,99 €","149,99 €",Zapatillas de training - Hombre,Materiales sostenibles,https://www.nike.com/es/t/metcon-1-og-zapatill...
7,Nike Metcon 9 AMP,"149,99 €","149,99 €",Zapatillas de training - Hombre,Materiales sostenibles,https://www.nike.com/es/t/metcon-9-amp-zapatil...
8,Nike Metcon 9 EasyOn,"97,99 €","139,99 €",Zapatillas de training - Mujer,Materiales sostenibles,https://www.nike.com/es/t/metcon-9-easyon-zapa...
9,Nike Metcon 9,"139,99 €","139,99 €",Zapatillas de training - Hombre,Materiales sostenibles,https://www.nike.com/es/t/metcon-9-zapatillas-...


In [ ]:
"""
REEBOK
import requests
import pandas as pd
import time

url2 = "https://www.reebok.eu/es-es/shopping/?query=nano&pageindex=1"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.google.com'
}
response2 = requests.get(url2,headers=headers)
from bs4 import BeautifulSoup
soup2=BeautifulSoup(response.content, "html.parser")
"""

In [ ]:
"""

REEBOK

import pandas as pd
from bs4 import BeautifulSoup
import time




def extract_product_data1(soup2):
    """
    Extrae los títulos y precios de los productos desde el HTML.
    
    Args:
        soup (BeautifulSoup): Objeto BeautifulSoup con el HTML parseado.

    Returns:
        pd.DataFrame: DataFrame con las columnas 'title' y 'price'.
    """
    # Crear DataFrame vacío
    df2 = pd.DataFrame(columns=['title', 'price'])

    # Extraer títulos y precios
    product_titles = soup.find_all('a', class_='css-6q57t3')
    product_prices = soup2.find_all('span', class_='css-gz3xcd')  # Extraer precios

    print(product_titles)

    items = min(len(product_titles), len(product_prices))

    # Iterar sobre las tarjetas y extraer información
    for i in range(items):
        # Pausa para evitar ser bloqueado por el servidor
        time.sleep(1)

        # Obtener el título del producto
        title = product_titles[i].text.strip()

        # Obtener el precio del producto
        price = product_prices[i].text.strip()

        # Agregar al DataFrame
        df2.loc[i] = [title, price]

    return df2
"""

In [ ]:
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Configuración de Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Abrir la página con Selenium
driver.get(url2)

# Esperar a que la página cargue (puedes aumentar el tiempo si es necesario)
time.sleep(5)  # Asegúrate de que la página esté completamente cargada

# Obtener el HTML final después de la ejecución de JavaScript
html = driver.page_source

# Parsear el HTML con BeautifulSoup
soup2 = BeautifulSoup(html, 'html.parser')

# Buscar los títulos de los productos
product_titles = soup2.find_all('a', class_='css-6q57t3')

# Imprimir los títulos
for title in product_titles:
    print(title.text.strip())

# Cerrar el navegador
driver.quit()
"""